####시나리오
사용자 정보에 저장된 사람들 중 출근한 사람들의 출근시간,퇴근시간,휴식한 시간 확인 가능한 프로그램
사용자 정보에 출근시간,퇴근시간,총 휴식시간 저장할 수 있게하는 함수 만들기
출근시간,퇴근시간 지정하기
사용자에게 보기 중에 어떤 정보를 입력받을건지 물어보는 함수 만들기 (출근,휴식시작,휴식종료,퇴근,근태관리,종료(종료 입력시 프로그램 종료))
사용자에게 이름 물어보기
출근을 입력받으면 현재 시간을 확인하여 현재 사용자의 이름에 출근시간 정보 저장하고 출근시간과 비교하여 정시,일찍 출근 시 정상출근하셨습니다 출력하기,지각 시 늦은 시간 (ex.지각:n분 지각) 출력하기
퇴근을 입력 받으면 현재 시간을 확인하여 현재 사용자의 이름에 퇴근시간 정보 저장하고 퇴근시간보다 이르면 정말 지금 퇴근하시겠습니까?(y/n)물어보고 y 입력하면 오늘도 고생하셨습니다 출력 퇴근시간과 같으면 오늘도 고생하셨습니다 출력 늦게 퇴근 시 오늘도 고생하셨습니다,연장근무:n분 출력
휴식시작 입력시 현재 시간 현재 사용자 정보에 저장해두기
휴식 종료 입력시 휴식 시작 시간보다 몇분이 지났는지 저장하고 출력하기
여러번 휴식 할 수 있게 반복 가능하게 만들기
근태관리 입력하면 전체 사용자의 출,퇴근,휴식시간 출력하기

In [2]:
#시간과 날짜 모듈 가져오기
import datetime
#근태관리를 위한 클래스 정의하기
class AttendanceManager:
  #클래스의 초기화 메서드를 정의하기
    def __init__(self):
        self.user_data = {} #사용자의 근태 정보를 저장하기 위한 빈 딕셔너리를 생성
        self.START_TIME = datetime.time(hour=9, minute=0)  # 출근 시간을 정의
        self.END_TIME = datetime.time(hour=18, minute=0)  # 퇴근 시간을 정의

    def save_attendance_info(self, name, date, start_time=None, end_time=None, break_times=None):
        # 사용자의 출근, 퇴근, 휴식시간을 저장합니다.
        if name not in self.user_data:
            self.user_data[name] = {}
        if date not in self.user_data[name]:
            self.user_data[name][date] = {'출근시간': start_time, '퇴근시간': end_time, '휴식시간': break_times}
        else:
            if start_time:
                self.user_data[name][date]['출근시간'] = start_time
            if end_time:
                self.user_data[name][date]['퇴근시간'] = end_time
            if break_times:
                self.user_data[name][date]['휴식시간'] = break_times

    def set_start_time(self, name):
        # 출근 시간을 설정합니다.
        now = datetime.datetime.utcnow() + datetime.timedelta(hours=9)  # UTC에서 9시간을 더하여 한국 시간으로 변경
        date = now.date().strftime('%Y-%m-%d')
        self.save_attendance_info(name, date, start_time=now)
        print(f"{name}님의 {date} 출근시간은 {now.strftime('%H:%M')}입니다.")

        if now.time() <= self.START_TIME:
            print("정상 출근하셨습니다.")
        else:
            minutes_late = (now - datetime.datetime.combine(now.date(), self.START_TIME)).seconds // 60
            print(f"지각: {minutes_late}분")
        print()

    def set_end_time(self, name):
        # 퇴근 시간을 설정합니다.
        now = datetime.datetime.utcnow() + datetime.timedelta(hours=9)  # UTC에서 9시간을 더하여 한국 시간으로 변경
        date = now.date().strftime('%Y-%m-%d')
        if self.user_data.get(name, {}).get(date, {}).get('출근시간') is None:
            print(f"{date} 출근을 먼저 기록해주세요.")
            return

        if now.time() < self.END_TIME:
            confirm = input("퇴근 시간보다 일찍 퇴근하시겠습니까? (y/n): ").strip().lower()
            if confirm != 'y':
                return

        self.save_attendance_info(name, date, end_time=now)
        print(f"{name}님의 {date} 퇴근시간은 {now.strftime('%H:%M')}입니다.")

        if now.time() < self.END_TIME:
            print("조기 퇴근하셨습니다.")
        else:
            overtime_minutes = (now - datetime.datetime.combine(now.date(), self.END_TIME)).seconds // 60
            print(f"연장근무: {overtime_minutes}분")
        print()

    def start_break(self, name):
        # 휴식 시작 시간을 설정합니다.
        now = datetime.datetime.utcnow() + datetime.timedelta(hours=9)  # UTC에서 9시간을 더하여 한국 시간으로 변경
        date = now.date().strftime('%Y-%m-%d')

        break_times = self.user_data.get(name, {}).get(date, {}).get('휴식시간')
        if break_times is None:
            break_times = [now]
        else:
            break_times.append(now)

        self.save_attendance_info(name, date, break_times=break_times)
        print(f"{name}님의 {date} 휴식이 시작되었습니다. ({now.strftime('%H:%M')})")

    def end_break(self, name):
        # 휴식 종료 시간을 설정합니다.
        now = datetime.datetime.utcnow() + datetime.timedelta(hours=9)  # UTC에서 9시간을 더하여 한국 시간으로 변경
        date = now.date().strftime('%Y-%m-%d')
        break_times = self.user_data.get(name, {}).get(date, {}).get('휴식시간')
        if break_times:
            break_times.append(now)
            self.save_attendance_info(name, date, break_times=break_times)
            duration = (now - break_times[-2]).seconds // 60
            print(f"{name}님은 {date}에 {duration}분 동안 휴식을 취했습니다.")
        else:
            print(f"{date} 휴식을 시작하지 않았습니다.")

    def print_user_data(self, name=None, date=None):
        # 특정 사용자의 특정 날짜의 출,퇴근,휴식시간을 출력합니다.
        if name:
            if date:
                if name in self.user_data and date in self.user_data[name]:
                    print(f"{name}님의 {date} 출,퇴근,휴식시간입니다:")
                    data = self.user_data[name][date]
                    print(f"출근시간 - {data.get('출근시간', '없음')}, "
                          f"퇴근시간 - {data.get('퇴근시간', '없음')}, ", end="")
                    if data.get('휴식시간'):
                        total_break_duration = sum([(break_time[1] - break_time[0]).seconds // 60 for break_time in zip(data['휴식시간'][::2], data['휴식시간'][1::2])])
                        print(f"휴식시간 - {total_break_duration}분")
                    else:
                        print("휴식시간 - 없음")
                    print()
                else:
                    print(f"{name}님의 {date} 정보가 없습니다.")
            else:
                print(f"{name}님의 출,퇴근,휴식시간입니다:")
                for d, data in self.user_data[name].items():
                    print(f"{d}: 출근시간 - {data.get('출근시간', '없음')}, "
                          f"퇴근시간 - {data.get('퇴근시간', '없음')}, ", end="")
                    if data.get('휴식시간'):
                        total_break_duration = sum([(break_time[1] - break_time[0]).seconds // 60 for break_time in zip(data['휴식시간'][::2], data['휴식시간'][1::2])])
                        print(f"휴식시간 - {total_break_duration}분")
                    else:
                        print("휴식시간 - 없음")
                    print()
        else:
            print("전체 사용자의 출,퇴근,휴식시간입니다:")
            if not self.user_data:
                print("등록된 사용자 정보가 없습니다.")
            else:
                for user, user_data in self.user_data.items():
                    print(f"{user}님의 정보:")
                    for d, data in user_data.items():
                        print(f"{d}: 출근시간 - {data.get('출근시간', '없음')}, "
                              f"퇴근시간 - {data.get('퇴근시간', '없음')}, ", end="")
                        if data.get('휴식시간'):
                            total_break_duration = sum([(break_time[1] - break_time[0]).seconds // 60 for break_time in zip(data['휴식시간'][::2], data['휴식시간'][1::2])])
                            print(f"휴식시간 - {total_break_duration}분")
                        else:
                            print("휴식시간 - 없음")
                        print()
            print()

    def manage_attendance(self):
        # 사용자가 선택한 기능에 따라 근태를 관리합니다.
        while True:
            print("사용할 기능을 선택해주세요:")
            action = input("출근, 퇴근, 휴식시작, 휴식종료, 근태관리, 종료: ").strip().lower()
            if action == '출근':
                name = input("사용자 이름을 입력하세요: ")
                self.set_start_time(name)
            elif action == '퇴근':
                name = input("사용자 이름을 입력하세요: ")
                self.set_end_time(name)
            elif action == '휴식시작':
                name = input("사용자 이름을 입력하세요: ")
                self.start_break(name)
            elif action == '휴식종료':
                name = input("사용자 이름을 입력하세요: ")
                self.end_break(name)
            elif action == '근태관리':
                user = input("사용자 이름을 입력하세요 (전체 사용자 정보를 보려면 엔터): ").strip()
                if user:
                    self.print_user_data(user)
                else:
                    self.print_user_data()
            elif action == '종료':
                print("프로그램을 종료합니다.")
                return
            else:
                print("올바른 기능을 선택해주세요.")

    def main(self):
        # 프로그램의 메인 함수입니다. 근태 관리를 시작합니다.
        self.manage_attendance()

if __name__ == "__main__":
    manager = AttendanceManager()
    manager.main()

사용할 기능을 선택해주세요:
출근, 퇴근, 휴식시작, 휴식종료, 근태관리, 종료: 출근
사용자 이름을 입력하세요: 김하준
김하준님의 2024-05-16 출근시간은 10:06입니다.
지각: 66분

사용할 기능을 선택해주세요:
출근, 퇴근, 휴식시작, 휴식종료, 근태관리, 종료: 휴식시작
사용자 이름을 입력하세요: 김하준
김하준님의 2024-05-16 휴식이 시작되었습니다. (10:07)
사용할 기능을 선택해주세요:
출근, 퇴근, 휴식시작, 휴식종료, 근태관리, 종료: 휴식종료
사용자 이름을 입력하세요: 김하준\
2024-05-16 휴식을 시작하지 않았습니다.
사용할 기능을 선택해주세요:
출근, 퇴근, 휴식시작, 휴식종료, 근태관리, 종료: 휴식종료
사용자 이름을 입력하세요: 김하준
김하준님은 2024-05-16에 0분 동안 휴식을 취했습니다.
사용할 기능을 선택해주세요:
출근, 퇴근, 휴식시작, 휴식종료, 근태관리, 종료: 출근
사용자 이름을 입력하세요: 김도원
김도원님의 2024-05-16 출근시간은 10:07입니다.
지각: 67분

사용할 기능을 선택해주세요:
출근, 퇴근, 휴식시작, 휴식종료, 근태관리, 종료: 근태관리
사용자 이름을 입력하세요 (전체 사용자 정보를 보려면 엔터): 
전체 사용자의 출,퇴근,휴식시간입니다:
김하준님의 정보:
2024-05-16: 출근시간 - 2024-05-16 10:06:35.016459, 퇴근시간 - None, 휴식시간 - 0분

김도원님의 정보:
2024-05-16: 출근시간 - 2024-05-16 10:07:46.296919, 퇴근시간 - None, 휴식시간 - 없음


사용할 기능을 선택해주세요:
출근, 퇴근, 휴식시작, 휴식종료, 근태관리, 종료: 퇴근
사용자 이름을 입력하세요: 홍길동
2024-05-16 출근을 먼저 기록해주세요.
사용할 기능을 선택해주세요:
출근, 퇴근, 휴식시작, 휴식종료, 근태관리, 종료: 퇴금
올바른 기능을 선택해주세요.
사용할 기능을 선택해주세요:
출근, 퇴근, 휴식시작, 휴식종료, 근태

In [ ]:
# 설명

import datetime
#import datetime: 파이썬의 내장 모듈인 datetime을 가져옵니다. 이 모듈을 사용하여 날짜 및 시간 정보를 다룰 수 있습니다.
class AttendanceManager:#근태 관리를 위한 AttendanceManager 클래스를 정의합니다.
    def __init__(self):  #AttendanceManager 클래스의 생성자 함수를 정의합니다.
        self.user_data = {} #사용자의 근태 정보를 저장하기 위한 빈 딕셔너리를 생성합니다.
        self.START_TIME = datetime.time(hour=9, minute=0)  # 출근 시간을 정의 #여기서는 오전 9시로 설정합니다.
        self.END_TIME = datetime.time(hour=18, minute=0)  # 퇴근 시간을 정의 #여기서는 오후 6시로 설정합니다.

    def save_attendance_info(self, name, start_time=None, end_time=None, break_time=None):
      #사용자의 근태 정보를 저장하는 메서드를 정의합니다.
        # 사용자의 출근, 퇴근, 휴식시간을 저장합니다.
        if name not in self.user_data:  #입력된 이름이 사용자 데이터에 없는 경우를 확인합니다.
            self.user_data[name] = {'출근시간': start_time, '퇴근시간': end_time, '휴식시간': break_time}
            #사용자 데이터에 새로운 사용자 정보를 추가합니다.
        else:  #입력된 이름이 사용자 데이터에 이미 있는 경우를 처리합니다.
            if start_time:  #만약 출근 시간이 주어졌다면, 해당 사용자의 출근 시간을 업데이트합니다.
                self.user_data[name]['출근시간'] = start_time
            if end_time:  #만약 퇴근 시간이 주어졌다면, 해당 사용자의 퇴근 시간을 업데이트합니다.
                self.user_data[name]['퇴근시간'] = end_time
            if break_time:  #만약 휴식 시간이 주어졌다면, 해당 사용자의 휴식 시간을 업데이트합니다.
                self.user_data[name]['휴식시간'] = break_time

    def set_start_time(self, name):  #출근 시간을 설정하는 메서드를 정의합니다.
        # 출근 시간을 설정합니다.
        now = datetime.datetime.now()  #현재 시간을 가져옵니다.
        self.save_attendance_info(name, start_time=now)  #save_attendance_info 메서드를 호출하여 출근 시간을 저장합니다.
        print(f"{name}님의 출근시간은 {now.strftime('%H:%M')}입니다.")

        if now.time() <= self.START_TIME:#현재 시간이 출근 시간보다 이르면 정상 출근 메시지를 출력합니다.
            print("정상 출근하셨습니다.")
        else:
            minutes_late = (now - datetime.datetime.combine(now.date(), self.START_TIME)).seconds // 60
            #지각한 경우, 지각 시간을 계산합니다.
            print(f"지각: {minutes_late}분")
        print()

    def set_end_time(self, name): #퇴근 시간을 설정하는 메서드를 정의합니다.
        # 퇴근 시간을 설정합니다.
        now = datetime.datetime.now()
        if self.user_data.get(name, {}).get('출근시간') is None:  #출근 기록이 없으면 퇴근할 수 없습니다.
            print("출근을 먼저 기록해주세요.")
            return

        self.save_attendance_info(name, end_time=now)  #퇴근 시간을 저장합니다.
        print(f"{name}님의 퇴근시간은 {now.strftime('%H:%M')}입니다.")

        if now.time() < self.END_TIME:    #퇴근 시간이 아직이면 조기 퇴근 메시지를 출력합니다.
            print("조기 퇴근하셨습니다.")
        else:
            overtime_minutes = (now - datetime.datetime.combine(now.date(), self.END_TIME)).seconds // 60
            #퇴근 시간이 지난 경우, 초과 근무 시간을 계산합니다.
            print(f"연장근무: {overtime_minutes}분")
        print()

    def start_break(self, name):  #사용자의 휴식을 시작하는 메서드를 정의합니다.
        # 휴식 시작 시간을 설정합니다.
        now = datetime.datetime.now()  #현재 시간을 가져옵니다.
        self.save_attendance_info(name, break_time=now)  #휴식 시작 시간을 저장합니다.
        print(f"{name}님의 휴식이 시작되었습니다. ({now.strftime('%H:%M')})") #휴식 시작 메시지를 출력합니다.

    def end_break(self, name):# 사용자의 휴식을 종료하는 메서드를 정의합니다.
        # 휴식 종료 시간을 설정합니다.
        now = datetime.datetime.now()
        break_start_time = self.user_data[name]['휴식시간']  #사용자의 휴식 시작 시간을 가져옵니다.
        if break_start_time: #만약 휴식을 시작한 시간이 존재한다면,
            pass  #현재는 아무 동작도 하지 않습니다. 여기서는 휴식 종료 시간을 설정하는 기능을 추가할 수 있습니다.
        else:  #만약 휴식을 시작하지 않았다면,
            print("휴식을 시작하지 않았습니다.")  #휴식을 시작하지 않았다는 메시지를 출력합니다.

    def print_user_data(self):  #모든 사용자의 근태 정보를 출력하는 메서드를 정의합니다.
        # 모든 사용자의 출근, 퇴근, 휴식시간을 출력합니다.
        print("전체 사용자의 출,퇴근,휴식시간입니다:")
        if not self.user_data:    #만약 사용자 데이터가 비어있다면,
            print("등록된 사용자 정보가 없습니다.")  #등록된 사용자 정보가 없다는 메시지를 출력합니다.
        else:
            for user, data in self.user_data.items():  #모든 사용자에 대해 반복문을 실행합니다.
                print(f"{user}: 출근시간 - {data.get('출근시간', '없음')}, "
                      f"퇴근시간 - {data.get('퇴근시간', '없음')}, ", end="")  #사용자의 출근 시간을 출력합니다. 출근 시간이 없을 경우 "없음"을 출력합니다.
                      #사용자의 퇴근 시간을 출력합니다. 퇴근 시간이 없을 경우 "없음"을 출력합니다.
                if data.get('휴식시간'):  #만약 사용자의 휴식 시간이 존재한다면,
                    break_duration = (datetime.datetime.now() - data['휴식시간']).seconds // 60
                    #휴식 기간을 분 단위로 계산합니다.
                    print(f"휴식시간 - {break_duration}분")  #휴식 시간을 출력합니다.
                else:
                    print("휴식시간 - 없음")
        print()

    def manage_attendance(self):
        # 사용자가 선택한 기능에 따라 근태를 관리합니다.
        while True:  # 무한 루프를 시작합니다.
            print("사용할 기능을 선택해주세요:")
            action = input("출근, 퇴근, 휴식시작, 휴식종료, 근태관리, 종료: ").strip().lower()
            if action == '출근':
                name = input("사용자 이름을 입력하세요: ")
                self.set_start_time(name)
            elif action == '퇴근':
                name = input("사용자 이름을 입력하세요: ")
                self.set_end_time(name)
            elif action == '휴식시작':
                name = input("사용자 이름을 입력하세요: ")
                self.start_break(name)
            elif action == '휴식종료':
                name = input("사용자 이름을 입력하세요: ")
                self.end_break(name)
            elif action == '근태관리':
                self.print_user_data()
            elif action == '종료':
                print("프로그램을 종료합니다.")
                return
            else:
                print("올바른 기능을 선택해주세요.")

    def main(self):  #프로그램의 메인 함수를 정의합니다.
        # 프로그램의 메인 함수입니다. 근태 관리를 시작합니다.
        self.manage_attendance()   #근태 관리를 시작하는 manage_attendance 메서드를 호출합니다.

if __name__ == "__main__":  #이 코드가 직접 실행될 때만 실행됩니다.
    manager = AttendanceManager()  #클래스의 인스턴스를 생성합니다.
    manager.main()   #메서드를 호출하여 프로그램을 실행합니다.

In [2]:

#시간과 날짜 모듈 가져오기
import datetime
import json
#근태관리를 위한 클래스 정의하기
class AttendanceManager:
    def __init__(self):
        self.user_data = {}
        self.START_TIME = datetime.time(hour=9, minute=0)
        self.END_TIME = datetime.time(hour=18, minute=0)
        self.data_file = "attendance_data.json"  # 데이터를 저장할 파일

    def save_data(self):
        # 사용자 데이터를 JSON 파일에 저장합니다.
        with open(self.data_file, "w") as file:
            json.dump(self.user_data, file)

    def load_data(self):
        # JSON 파일에서 사용자 데이터를 로드합니다.
        try:
            with open(self.data_file, "r") as file:
                self.user_data = json.load(file)
        except FileNotFoundError:
            # 파일이 없는 경우 무시하고 진행합니다.
            pass

    def save_attendance_info(self, name, date, start_time=None, end_time=None, break_times=None):
        # 사용자의 출근, 퇴근, 휴식시간을 저장합니다.
        if name not in self.user_data:
            self.user_data[name] = {}
        if date not in self.user_data[name]:
            self.user_data[name][date] = {'출근시간': start_time, '퇴근시간': end_time, '휴식시간': break_times}
        else:
            if start_time:
                self.user_data[name][date]['출근시간'] = start_time
            if end_time:
                self.user_data[name][date]['퇴근시간'] = end_time
            if break_times:
                self.user_data[name][date]['휴식시간'] = break_times


    def set_start_time(self, name):
        # 출근 시간을 설정합니다.
        now = datetime.datetime.utcnow() + datetime.timedelta(hours=9)  # UTC에서 9시간을 더하여 한국 시간으로 변경
        date = now.date().strftime('%Y-%m-%d')
        self.save_attendance_info(name, date, start_time=now)
        print(f"{name}님의 {date} 출근시간은 {now.strftime('%H:%M')}입니다.")

        if now.time() <= self.START_TIME:
            print("정상 출근하셨습니다.")
        else:
            minutes_late = (now - datetime.datetime.combine(now.date(), self.START_TIME)).seconds // 60
            print(f"지각: {minutes_late}분")
        print()

    def set_end_time(self, name):
        # 퇴근 시간을 설정합니다.
        now = datetime.datetime.utcnow() + datetime.timedelta(hours=9)  # UTC에서 9시간을 더하여 한국 시간으로 변경
        date = now.date().strftime('%Y-%m-%d')
        if self.user_data.get(name, {}).get(date, {}).get('출근시간') is None:
            print(f"{date} 출근을 먼저 기록해주세요.")
            return

        if now.time() < self.END_TIME:
            confirm = input("퇴근 시간보다 일찍 퇴근하시겠습니까? (y/n): ").strip().lower()
            if confirm != 'y':
                return

        self.save_attendance_info(name, date, end_time=now)
        print(f"{name}님의 {date} 퇴근시간은 {now.strftime('%H:%M')}입니다.")

        if now.time() < self.END_TIME:
            print("조기 퇴근하셨습니다.")
        else:
            overtime_minutes = (now - datetime.datetime.combine(now.date(), self.END_TIME)).seconds // 60
            print(f"연장근무: {overtime_minutes}분")
        print()

    def start_break(self, name):
        # 휴식 시작 시간을 설정합니다.
        now = datetime.datetime.utcnow() + datetime.timedelta(hours=9)  # UTC에서 9시간을 더하여 한국 시간으로 변경
        date = now.date().strftime('%Y-%m-%d')

        break_times = self.user_data.get(name, {}).get(date, {}).get('휴식시간')
        if break_times is None:
            break_times = [now]
        else:
            break_times.append(now)

        self.save_attendance_info(name, date, break_times=break_times)
        print(f"{name}님의 {date} 휴식이 시작되었습니다. ({now.strftime('%H:%M')})")

    def end_break(self, name):
        # 휴식 종료 시간을 설정합니다.
        now = datetime.datetime.utcnow() + datetime.timedelta(hours=9)  # UTC에서 9시간을 더하여 한국 시간으로 변경
        date = now.date().strftime('%Y-%m-%d')
        break_times = self.user_data.get(name, {}).get(date, {}).get('휴식시간')
        if break_times:
            break_times.append(now)
            self.save_attendance_info(name, date, break_times=break_times)
            duration = (now - break_times[-2]).seconds // 60
            print(f"{name}님은 {date}에 {duration}분 동안 휴식을 취했습니다.")
        else:
            print(f"{date} 휴식을 시작하지 않았습니다.")

    def print_user_data(self, name=None, date=None):
        # 특정 사용자의 특정 날짜의 출,퇴근,휴식시간을 출력합니다.
        if name:
            if date:
                if name in self.user_data and date in self.user_data[name]:
                    print(f"{name}님의 {date} 출,퇴근,휴식시간입니다:")
                    data = self.user_data[name][date]
                    print(f"출근시간 - {data.get('출근시간', '없음')}, "
                          f"퇴근시간 - {data.get('퇴근시간', '없음')}, ", end="")
                    if data.get('휴식시간'):
                        total_break_duration = sum([(break_time[1] - break_time[0]).seconds // 60 for break_time in zip(data['휴식시간'][::2], data['휴식시간'][1::2])])
                        print(f"휴식시간 - {total_break_duration}분")
                    else:
                        print("휴식시간 - 없음")
                    print()
                else:
                    print(f"{name}님의 {date} 정보가 없습니다.")
            else:
                print(f"{name}님의 출,퇴근,휴식시간입니다:")
                for d, data in self.user_data[name].items():
                    print(f"{d}: 출근시간 - {data.get('출근시간', '없음')}, "
                          f"퇴근시간 - {data.get('퇴근시간', '없음')}, ", end="")
                    if data.get('휴식시간'):
                        total_break_duration = sum([(break_time[1] - break_time[0]).seconds // 60 for break_time in zip(data['휴식시간'][::2], data['휴식시간'][1::2])])
                        print(f"휴식시간 - {total_break_duration}분")
                    else:
                        print("휴식시간 - 없음")
                    print()
        else:
            print("전체 사용자의 출,퇴근,휴식시간입니다:")
            if not self.user_data:
                print("등록된 사용자 정보가 없습니다.")
            else:
                for user, user_data in self.user_data.items():
                    print(f"{user}님의 정보:")
                    for d, data in user_data.items():
                        print(f"{d}: 출근시간 - {data.get('출근시간', '없음')}, "
                              f"퇴근시간 - {data.get('퇴근시간', '없음')}, ", end="")
                        if data.get('휴식시간'):
                            total_break_duration = sum([(break_time[1] - break_time[0]).seconds // 60 for break_time in zip(data['휴식시간'][::2], data['휴식시간'][1::2])])
                            print(f"휴식시간 - {total_break_duration}분")
                        else:
                            print("휴식시간 - 없음")
                        print()
            print()

    def manage_attendance(self):
        # 사용자가 선택한 기능에 따라 근태를 관리합니다.
        while True:
            print("사용할 기능을 선택해주세요:")
            action = input("출근, 퇴근, 휴식시작, 휴식종료, 근태관리, 종료: ").strip().lower()
            if action == '출근':
                name = input("사용자 이름을 입력하세요: ")
                self.set_start_time(name)
            elif action == '퇴근':
                name = input("사용자 이름을 입력하세요: ")
                self.set_end_time(name)
            elif action == '휴식시작':
                name = input("사용자 이름을 입력하세요: ")
                self.start_break(name)
            elif action == '휴식종료':
                name = input("사용자 이름을 입력하세요: ")
                self.end_break(name)
            elif action == '근태관리':
                user = input("사용자 이름을 입력하세요 (전체 사용자 정보를 보려면 엔터): ").strip()
                if user:
                    self.print_user_data(user)
                else:
                    self.print_user_data()
            elif action == '종료':
                print("프로그램을 종료합니다.")
                return
            else:
                print("올바른 기능을 선택해주세요.")

    def main(self):
        # 프로그램의 메인 함수입니다. 근태 관리를 시작합니다.
        self.manage_attendance()

if __name__ == "__main__":
    manager = AttendanceManager()
    manager.main()

사용할 기능을 선택해주세요:
출근, 퇴근, 휴식시작, 휴식종료, 근태관리, 종료: 출근
사용자 이름을 입력하세요: 김하준
김하준님의 2024-05-16 출근시간은 16:49입니다.
지각: 469분

사용할 기능을 선택해주세요:
출근, 퇴근, 휴식시작, 휴식종료, 근태관리, 종료: 휴식시작
사용자 이름을 입력하세요: 김하준
김하준님의 2024-05-16 휴식이 시작되었습니다. (16:49)
사용할 기능을 선택해주세요:
출근, 퇴근, 휴식시작, 휴식종료, 근태관리, 종료: 휴식종료
사용자 이름을 입력하세요: 김하준
김하준님은 2024-05-16에 0분 동안 휴식을 취했습니다.
사용할 기능을 선택해주세요:
출근, 퇴근, 휴식시작, 휴식종료, 근태관리, 종료: 퇴근
사용자 이름을 입력하세요: 김하준
퇴근 시간보다 일찍 퇴근하시겠습니까? (y/n): y
김하준님의 2024-05-16 퇴근시간은 16:49입니다.
조기 퇴근하셨습니다.

사용할 기능을 선택해주세요:
출근, 퇴근, 휴식시작, 휴식종료, 근태관리, 종료: 근태관리
사용자 이름을 입력하세요 (전체 사용자 정보를 보려면 엔터): 
전체 사용자의 출,퇴근,휴식시간입니다:
김하준님의 정보:
2024-05-16: 출근시간 - 2024-05-16 16:49:20.654848, 퇴근시간 - 2024-05-16 16:49:40.195155, 휴식시간 - 0분


사용할 기능을 선택해주세요:
출근, 퇴근, 휴식시작, 휴식종료, 근태관리, 종료: 종료
프로그램을 종료합니다.
